In [1]:
import numpy as np
from scipy.io import loadmat
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.pyplot import cm
import matplotlib as mpl
import cv2
import computer_vision as cv
from icecream import ic
from tqdm import trange
from numba import njit

%load_ext snakeviz
# %matplotlib inline
%matplotlib qt
%config InlineBackend.figure_format = 'retina'
from matplotlib import rc
rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
rc('text', usetex=True)

In [2]:
def compute_sift_points(img1, img2, marg):
    sift = cv2.SIFT_create()
    kp1, des1 = sift.detectAndCompute(img1, None)
    kp2, des2 = sift.detectAndCompute(img2, None)

    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
    search_params = dict(checks=50)   # or pass empty dictionary

    flann = cv2.FlannBasedMatcher(index_params, search_params)
    matches = flann.knnMatch(des1, des2, k=2)

    print('Number of matches:', np.size(matches,0))

    good_matches = []
    for m, n in matches:
        if m.distance < marg*n.distance:
            good_matches.append([m])

    draw_params = dict(matchColor=(255,0,255), singlePointColor=(0,255,0), matchesMask=None, flags=cv2.DrawMatchesFlags_DEFAULT)
    img_match = cv2.drawMatchesKnn(img1, kp1, img2, kp2, good_matches, None, **draw_params)

    x1 = np.stack([kp1[match[0].queryIdx].pt for match in good_matches],1)
    x2 = np.stack([kp2[match[0].trainIdx].pt for match in good_matches],1)

    x1 = cv.homogenize(x1, multi=True)
    x2 = cv.homogenize(x2, multi=True)

    print('Number of good matches:', np.size(x1,1))

    return x1, x2, img_match

def get_sift_plot_points(img1_pts, img2_pts, img1):
    x = [img1_pts[0,:], np.size(img1,1)+img2_pts[0,:]]
    y = [img1_pts[1,:], img2_pts[1,:]]
    return x, y

def plot_sift_points(x1, x2, img1, img_match, path, save=False):
    x, y = get_sift_plot_points(x1, x2, img1)

    fig = plt.figure(figsize=(18,9))
    ax = plt.axes()

    ax.plot(x, y, 'o-', ms=5, lw=1, color='magenta')
    ax.set_xlabel('$x$')
    ax.set_ylabel('$y$')
    ax.set_aspect('equal')
    # ax.legend(loc="upper right")
    ax.imshow(cv2.cvtColor(img_match, cv2.COLOR_BGR2RGB))
    fig.tight_layout()
    if save:
        fig.savefig(path, dpi=300)
    plt.show()

In [3]:
def compute_ransac_iterations(alpha, epsilon, s):
    T = np.ceil(np.log(1-alpha) / np.log(1-epsilon**s))
    return T

In [4]:
def estimate_E_robust(K, x1_norm, x2_norm, n_its, n_samples, err_threshold_px):
    
    err_threshold = err_threshold_px / K[0,0]
    best_inliers = None
    best_E = None
    max_inliers = 0

    for t in trange(n_its):

        rand_mask = np.random.choice(np.size(x1_norm,1), n_samples, replace=False)
        E = cv.estimate_E_DLT(x1_norm[:,rand_mask], x2_norm[:,rand_mask], enforce=True, verbose=False)

        D1, D2 = cv.compute_epipolar_errors(E, x1_norm, x2_norm)
        inliers = ((D1**2 + D2**2) / 2) < err_threshold**2

        n_inliers = np.sum(inliers)

        if n_inliers > max_inliers:
            best_inliers = np.copy(inliers)
            best_E = np.copy(E)
            max_inliers = n_inliers

            print(np.sum(inliers), end='\r')
        
    return best_E, best_inliers

In [5]:
def plot_cameras_and_3D_points(X, C_arr, axis_arr, s, path, save=False):
    
    fig = plt.figure(figsize=(8,6))
    ax = plt.axes(projection='3d')

    ax.plot(X[0], X[1], X[2], '.', ms=0.9, color='magenta', label='3D points')
    cv.plot_cameras_and_axes(ax, C_arr, axis_arr, s)

    ax.set_xlabel('$x$')
    ax.set_ylabel('$y$')
    ax.set_zlabel('$z$')
    ax.axis('equal')
    # ax.view_init(elev=-50, azim=-104, roll=60)
    plt.legend(loc="lower right")
    if save:
        fig.savefig(path, dpi=300)
    plt.show()

In [6]:
path = r'C:\Users\erikn\skola\EEN020-Computer-Vision\assignment-4'
data = r'\data-2023\data'
compex = r'\compEx2data.mat'
img1 = r'\fountain1.png'
img2 = r'\fountain2.png'
report = r'\report-images'

fountain1 = cv.load_image(path+data+img1)
fountain2 = cv.load_image(path+data+img2)
K = cv.convert_mat_to_np(path+data+compex, 'K')
K_inv = np.linalg.inv(K)

ransac = True
plt_3D = True
save = True
snakeviz = False
ind = 3

In [47]:
marg = 0.7
x1, x2, fountain_match = compute_sift_points(fountain1, fountain2, marg)
# plot_sift_points(x1, x2, fountain1, fountain_match, path+report+'/CE2_sift.png', save=True)

x1_norm = cv.transform_and_dehomogenize(K_inv, x1)
x2_norm = cv.transform_and_dehomogenize(K_inv, x2)

np.save(path+data+'/x1_norm_ce2_{}.npy'.format(ind), x1_norm)
np.save(path+data+'/x2_norm_ce2_{}.npy'.format(ind), x2_norm)

Number of matches: 4381
Number of good matches: 170
(3, 170) (3, 170)


In [48]:
if ransac:
    n_its = 100000
    n_samples = 8
    err_threshold_px = 5
    E, inliers = estimate_E_robust(K, x1_norm, x2_norm, n_its, n_samples, err_threshold_px)
    
    np.save(path+data+'/E_robust_ce2_{}.npy'.format(ind), E)
    np.save(path+data+'/inliers_ce2_{}.npy'.format(ind), inliers)
    print('No. inliers:', np.sum(inliers))
    
elif snakeviz:
    %snakeviz estimate_E_robust(K, x1_norm, x2_norm)
else:
    E = cv.estimate_E_DLT(x1_norm, x2_norm, enforce=True, verbose=False)

  0%|          | 88/100000 [00:00<01:53, 877.72it/s]

  0%|          | 484/100000 [00:00<01:45, 947.50it/s]

  2%|▏         | 2437/100000 [00:02<01:47, 910.53it/s] 

 15%|█▍        | 14836/100000 [00:13<01:17, 1105.14it/s]

100%|██████████| 100000/100000 [01:49<00:00, 913.53it/s]

No. inliers: 140


In [15]:
if ransac:
    ind = 3
    E = np.load(path+data+'/E_robust_ce2_{}.npy'.format(ind))
    inliers = np.load(path+data+'/inliers_ce2_{}.npy'.format(ind))
    x1_norm = np.load(path+data+'/x1_norm_c2_{}.npy'.format(ind))
    x2_norm = np.load(path+data+'/x2_norm_c2_{}.npy'.format(ind))

    x1_norm = x1_norm[:,inliers]
    x2_norm = x2_norm[:,inliers]

F = cv.convert_E_to_F(E, K, K)

print('\nE:', E)
print('F:', F)

P2_arr = cv.extract_P_from_E(E)
P1 = cv.get_canonical_camera()

X_arr_dh = np.array([cv.dehomogenize(cv.triangulate_3D_point_DLT(P1, P2, x1_norm, x2_norm, print_svd=False)) for P2 in P2_arr])
P2_valid, X_valid = cv.extract_valid_camera_and_points(P1, P2_arr, X_arr_dh)

i = 3
X_valid = X_arr_dh[i]
P2_valid = P2_arr[i]

if plt_3D:
    P1 = cv.transform(K, P1)
    P2 = cv.transform(K, P2_valid)

    P_arr = np.array([P1, P2])
    C_arr, axis_arr = cv.compute_camera_and_normalized_principal_axis(P_arr, multi=True)

    s = 1
    plot_cameras_and_3D_points(X_valid, C_arr, axis_arr, s, path+report+'/CE2_3D_P2_{}.png'.format(i+1), save)

# if plt_3D:
#     for i in range(4):
#         P2 = P_arr[i]
#         X = X_arr_dh[i]

#         P1 = cv.transform(K, P1)
#         P2 = cv.transform(K, P2)
            
#         P_arr = np.array([P1, P2])
#         C_arr, axis_arr = cv.compute_camera_and_normalized_principal_axis(P_arr, multi=True)

#         s = 10
#         plot_cameras_and_3D_points(X, C_arr, axis_arr, s, path+report+'/CE2_3D_P2_{}.png'.format(i+1), save)


E: [[  0.16170779   6.12810554  -0.49704611]
 [  9.60676748   0.56788438  14.76907712]
 [ -0.65269161 -16.47799254   1.        ]]
F: [[ 3.83404093e-09  1.45049461e-07 -1.84387576e-04]
 [ 2.27387802e-07  1.34188062e-08  6.05355064e-04]
 [-2.77469966e-04 -1.31035595e-03  1.00000000e+00]]
[1.76283187e+01 1.76283187e+01 4.42274025e-16]
No. valid coords for each camera pair: [140 140 280   0]
Argmax(P2_arr): [140 140 280   0]


In [50]:
# All points:
# Ransac: False
# Mean distance 1: 61.33360616690827
# Mean distance 2: 63.46892741628846
# RMS error: 155.95832290161417

# Inliers:
# Ransac: False
# Mean distance 1: 14.527720653307188
# Mean distance 2: 15.349401111343274
# RMS error: 21.875573131094626
# No. inliers: 5231

# All points:
# Ransac: True
# Mean distance 1: 58.74538197965829
# Mean distance 2: 71.52507914842866
# RMS error: 209.63596816010937

# Inliers:
# Ransac: True
# Mean distance 1: 0.8859416907684401
# Mean distance 2: 1.0020265668647925
# RMS error: 1.084528021008188
# No. inliers: 5231